## SQLite - Connecting to SQLite database

In [9]:
DB_FILE_NAME = "__FILES/companies.db"

In [2]:
import sqlite3

conn = sqlite3.connect(DB_FILE_NAME)

crs = conn.cursor()

# use the database here

conn.close()

## Create TABLE in SQLite

In [10]:
import sqlite3

conn = sqlite3.connect(DB_FILE_NAME)
crs = conn.cursor()

sql = '''
    CREATE TABLE companies (
                    id          INTEGER PRIMARY KEY AUTOINCREMENT,
                    name        VARCRCHAR(100) UNIQUE NOT NULL,
                    established INTEGER NOT NULL,
                    employees   INTEGER DEFAULT 0
)
'''

try:
    crs.execute(sql)
except sqlite3.OperationalError as err:
    print(f'sqlite error: {err.args[0]}')  # table companies already exists
    print(f'remove companies.db to show how it works')

conn.commit()

conn.close()

print('done')

sqlite error: table companies already exists
remove companies.db to show how it works
done


## INSERT data into SQLite database


In [11]:
import sqlite3

sql = 'INSERT INTO companies (name, employees, established) VALUES (?, ?, ?)'

def insert_one(conn, crs):
    company_name = 'Hostlocal'
    employee_count = 1
    year_of_establishment = 2000

    try:
        crs.execute(sql, (company_name, employee_count, year_of_establishment))
    except sqlite3.IntegrityError as err:
        print('sqlite error: ', err.args[0]) # column name is not unique
    conn.commit()


def insert_many(conn, crs):
    companies = [
        ('Google',    150_028, 1998),
        ('Facebook',   68_177, 2003),
        ('Apple',     154_000, 1977),
        ('Microsoft', 181_000, 1975),
    ]

    try:
        crs.executemany(sql, companies)
    except sqlite3.IntegrityError as err:
        print(f'sqlite error: {err.args[0]}')
    conn.commit()


def main():
    conn = sqlite3.connect(DB_FILE_NAME)
    crs = conn.cursor()
    insert_one(conn, crs)
    insert_many(conn, crs)
    conn.close()
    print('done')

main()

done


## SELECT data from SQLite database


In [13]:
import sqlite3

conn = sqlite3.connect(DB_FILE_NAME)
crs = conn.cursor()

employees = 3

sql = 'SELECT * FROM companies WHERE employees >= ?'
for company in crs.execute(sql, (employees,)):
    print(company)

print('-----------')

year = 2000
sql = 'SELECT id, name FROM companies WHERE employees >= ? AND established < ?'
for id, name in crs.execute(sql, (employees, year)):
    print(f"{id} - {name}")

conn.close()

(2, 'Google', 1998, 150028)
(3, 'Facebook', 2003, 68177)
(4, 'Apple', 1977, 154000)
(5, 'Microsoft', 1975, 181000)
-----------
2 - Google
4 - Apple
5 - Microsoft


## SELECT aggregate data from SQLite database


- If expecting only one row, call the fetchone method. If the result set might be empty, then the fetchone might return None. Check for it!

In [15]:
import sqlite3

conn = sqlite3.connect(DB_FILE_NAME)
crs = conn.cursor()

employees = 3
year = 2000

sql = 'SELECT COUNT(id) FROM companies WHERE employees >= ? AND established < ?'
crs.execute(sql, (employees, year))
row = crs.fetchone()
print(row)
print(row[0])

name = '%o%'
sql = 'SELECT SUM(employees) FROM companies WHERE name LIKE ? AND established < ?'
crs.execute(sql, (name, year))
row = crs.fetchone()
print(row)
print(row[0])


conn.close()

(3,)
3
(331028,)
331028


## SELECT data from SQLite database into dictionaries


In [16]:
import sqlite3

conn = sqlite3.connect(DB_FILE_NAME)
conn.row_factory = sqlite3.Row

crs = conn.cursor()

employees = 0
year = 2000

sql = 'SELECT * FROM companies WHERE employees >= ? AND established < ?'
for company in crs.execute(sql, (employees, year)):
    # returns sqlite3.Row objects, but they can also act as dictionaries
    print(f"{company['id']} - {company['name']} - {company['employees']} - {company['established']}")

conn.close()

2 - Google - 150028 - 1998
4 - Apple - 154000 - 1977
5 - Microsoft - 181000 - 1975


## UPDATE data in SQLite database


- UPDATE works quite similar, but it might have a WHERE clause.


In [17]:

conn = sqlite3.connect(DB_FILE_NAME)
crs = conn.cursor()

name = 'Hostlocal'

sql = 'SELECT employees FROM companies WHERE name = ?'
crs.execute(sql, (name,))
row = crs.fetchone()
employees = row[0]


sql = 'UPDATE companies SET employees=? WHERE name = ?'
crs.execute(sql, (employees+1, name))
conn.commit()

print('-----------')

sql = 'SELECT name, employees FROM companies'
for name, employees in crs.execute(sql):
    print(f"{name} - {employees}")


conn.close()

-----------
Hostlocal - 2
Google - 150028
Facebook - 68177
Apple - 154000
Microsoft - 181000
